# Pre-trained models without head 

In [9]:
# Text classification using BERT 

In [15]:
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer
import torch

model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

text ="I am feeling happy"

class SimpleClassifier(nn.Module):
    def __init__(self, input_size, num_classes):
        super(SimpleClassifier, self).__init__()
        self.fc = nn.Linear(input_size, num_classes)
    def forward(self, x):
        return self.fc(x)
print('ready')


ready


In [16]:
# trying the model

In [17]:
inputs = tokenizer(text, return_tensors="pt",padding=True,
                   truncation=True,max_length=64)

outputs = model(**inputs)

pooled_output = outputs.pooler_output
print("hidden states size: ",outputs.last_hidden_state.shape)
print('pooled output size: ', pooled_output.shape)

classifier_head = SimpleClassifier(pooled_output.size(-1), num_classes=2)
logits = classifier_head(pooled_output)
probs= torch.softmax(logits, dim=1)
print("predicted class probabilities: ", probs)

hidden states size:  torch.Size([1, 6, 768])
pooled output size:  torch.Size([1, 768])
predicted class probabilities:  tensor([[0.5192, 0.4808]], grad_fn=<SoftmaxBackward0>)


# pre-trained model with defined head

In [18]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

In [23]:
# define model
model_name="nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# run model on text
text = "today I scored 5 goals during the soccer game"
inputs = tokenizer(text,return_tensors="pt")
outputs=model(**inputs)
logits = outputs.logits

# predicted class
predicted_class = torch.argmax(logits,dim=1).item()
print(f"Predicted class index: {predicted_class + 1} star.")


Predicted class index: 5 star.


# Text generation with defined head

In [25]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [27]:
model_name ="gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

promt ="Today I play soccer and I score "
inputs = tokenizer.encode(promt, return_tensors="pt")
output = model.generate(inputs,max_length=26)

generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print("Generated tet:")
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated tet:
Today I play soccer and I score  a lot of goals. I'm a big fan of the game and I'm a


# dataset for text classification

In [38]:
!pip install datasets==2.15.0

   ---------------------------------------- 0.0/521.2 kB ? eta -:--:--
   ------- -------------------------------- 92.2/521.2 kB 1.7 MB/s eta 0:00:01
   ---------------------------- ----------- 368.6/521.2 kB 3.8 MB/s eta 0:00:01
   ---------------------------------------- 521.2/521.2 kB 4.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/166.4 kB ? eta -:--:--
   ---------------------------------------- 166.4/166.4 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.3.1
    Uninstalling fsspec-2024.3.1:
      Successfully uninstalled fsspec-2024.3.1
  Attempting uninstall: datasets
    Found existing installation: datasets 2.20.0
    Uninstalling datasets-2.20.0:
      Successfully uninstalled datasets-2.20.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
s3fs 2024.3.1 requires fsspec==2024.3.1, but you have fsspec 2023.10.0 which is incompatible.

[notice] A new release of pip is available: 24.1 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [39]:
from datasets import load_dataset
from torch.utils.data import DataLoader

In [41]:
dataset = load_dataset("imdb")

ValueError: Invalid pattern: '**' can only be an entire path component

In [43]:
#!pip uninstall datasets


In [44]:
!pip install datasets==2.15.0


[notice] A new release of pip is available: 24.1 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [45]:
import datasets
print(datasets.__version__)

2.11.0


In [47]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name="t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

input_ids = tokenizer.encode("summarize: " + example["Article"],return_tensors="pt",
                             max_length=512, truncation=True)

summary_ids = model.generate(input_ids, max_length=150)
summary = tokenizer.decode(summary_ids[0],skip_special_tokens=True)

print("Original text")
print(example["Article"])
print("\nGenerated Summary:")
print(summary)

NameError: name 'example' is not defined

In [48]:
# exploring translation datasets

In [49]:
from datasets import load_dataset

dataset = load_dataset("techiaith/legislation-gov-uk_en_cy")

sample_data = dataset["train"]

input_example = sample_data.data['source'][0]
target_example = sample_data.data['target'][0]
print("input(english):",input_example)
print("target(welsh):", target_example)

FileNotFoundError: Couldn't find a dataset script at c:\Users\bciez\Documents\projects\LLMs\techiaith\legislation-gov-uk_en_cy\legislation-gov-uk_en_cy.py or any data file in the same directory. Couldn't find 'techiaith/legislation-gov-uk_en_cy' on the Hugging Face Hub either: FileNotFoundError: Dataset 'techiaith/legislation-gov-uk_en_cy' doesn't exist on the Hub. If the repo is private or gated, make sure to log in with `huggingface-cli login`.

# Loading a pre-trained LLM for translation

In [51]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "Helsinki-NLP/opus-mt-en-cy"
tokenizer= AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

input_seq ="2 regulations uncer section 1: Supplementary"
input_ids = tokenizer.encode(input_seq, return_tensors="pt")
translated_ids = model.generate(input_ids)
translated_text = tokenizer.decode(translated_ids[0], skip_special_tokens=True)
print("Predicted (Welch):", translated_text)

c:\Users\bciez\anaconda3\lib\site-packages\transformers\models\marian\tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Predicted (Welch): 2 [c]


In [52]:
model_name = "Helsinki-NLP/opus-mt-en-es"

# Load the tokenizer and the model checkpoint
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

english_inputs = ["Hello", "Thank you", "How are you?", "Sorry", "Goodbye"]

# Encode the inputs, generate translations, decode, and print them
for english_input in english_inputs:
    input_ids = tokenizer.encode(english_input, return_tensors="pt")
    translated_ids = model.generate(input_ids)
    translated_text = tokenizer.decode(translated_ids[0], skip_special_tokens=True)
    print(f"English: {english_input} | Spanish: {translated_text}")

English: Hello | Spanish: Hola.
English: Thank you | Spanish: Gracias.
English: How are you? | Spanish: ¿Cómo estás?
English: Sorry | Spanish: Lo siento.
English: Goodbye | Spanish: Adiós.


In [53]:
# Exporing a QA dataset

In [54]:
from datasets import load_dataset
mlqa = load_dataset("xtreme",name="MLQA.en.en")
print(mlqa)

print("question:",mlqa['test']['question'][53])
print('answer:',mlqa['test']['answers'][53])
print('context:', mlqa['test']['context'][53])

ValueError: Invalid pattern: '**' can only be an entire path component

In [55]:

# Extractive QA: tokenizing inputs 

from transformers import AutoTokenizer


model_ckp = "deepset/minilm-uncased-squad2"
tokenizer = AutoTokenizer.from_pretrained(model_ckp)

question= "How was the game today?"
context="""Today I played soocer, and I scored 5 goals and I was very happy about it"""
inputs = tokenizer(question, context, return_tensors="pt")

from transformers import AutoModelForQuestionAnswering

model = AutoModelForQuestionAnswering.from_pretrained(model_ckp)

with torch.no_grad():
    outputs = model(**inputs)

start_idx = torch.argmax(outputs.start_logits)
end_idx = torch.argmax(output.end_logits) + 1

answer_span = inputs["input_ids"][0][start_idx:end_idx]
answer = tokenizer.decode(answer_span)


tokenizer_config.json:   0%|          | 0.00/107 [00:00<?, ?B/s]

c:\Users\bciez\anaconda3\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\bciez\.cache\huggingface\hub\models--deepset--minilm-uncased-squad2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
c:\Users\bciez\anaconda3\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_downl

config.json:   0%|          | 0.00/477 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

Some weights of the model checkpoint at deepset/minilm-uncased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


AttributeError: 'Tensor' object has no attribute 'end_logits'

In [ ]:
# for long context sequences

In [56]:
long_exmp = tokenizer(example_qt, example_ct, return_overflowing_tokens=True,
                      max_length=100, stride=25)


for idx, window in enumerate(long_exmp['input_ids']):
    print("tokens in window", idx," : ", len(window))

NameError: name 'example_qt' is not defined

In [57]:
for window in long_exmp['inputs_ids']:
    print(tokenizer.decode(window), "\n")

NameError: name 'long_exmp' is not defined

In [58]:
44903416 - 1000000

43903416

In [59]:
44903416 + 1000000

45903416